In [73]:
%matplotlib qt
#%matplotlib

import numpy as np
import cv2 as cv
print(f'Version de OpenCV: {cv.__version__}')
import matplotlib.pyplot as plt
import os
import glob
from sklearn.cluster import DBSCAN

Version de OpenCV: 4.7.0


### Punto 2

(4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen coca multi.png con el mismo témplate del item 1

In [ ]:
# Cargar imagen y template
img2_color = cv.imread('./images/coca_multi.png')
img2_gray = cv.cvtColor(img2_color, cv.COLOR_BGR2GRAY)

img1 = cv.imread('./template/pattern.png', 0)  # Template original
img1 = cv.resize(img1, (0, 0), fx=0.5, fy=0.5, interpolation=cv.INTER_AREA)

# Aplicar suavizado gaussiano
img1 = cv.GaussianBlur(img1, (3, 3), 0)
img2 = cv.GaussianBlur(img2_gray, (3, 3), 0)

# Aplicar Canny luego del suavizado
#img1 = cv.Canny(img1, 200, 200)
img2 = cv.Canny(img2, 200, 200)
img1 = 255 - img1.copy() 
# Mostrar resultados finales
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.imshow(img1, cmap='gray')
plt.title("Template (Gaussian + Canny)")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(img2, cmap='gray')
plt.title("Imagen (Gaussian + Canny)")
plt.axis('off')

plt.show()

In [26]:
# Creamos el vector de características SIFT

sift = cv.xfeatures2d.SIFT_create(nfeatures=5000, contrastThreshold=0.01)


# Y buscamos según el algoritmo...
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

print(len(kp1))
print(len(kp2))

70
5000


In [32]:
# parámetros FLANN
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 150)
search_params = dict(checks=200)   # o pasar un diccionario vacío
flann = cv.FlannBasedMatcher(index_params,search_params)
# K es la cantidad de "best matches" para cada descriptor
matches = flann.knnMatch(des1, des2, k=2)

# Necesidad de dibujar solo las coincidencias buenas, se crea una máscara
matchesMask = [[0,0] for i in range(len(matches))]

# Ratio de verificación como figura en el paper de Lowe
for i,(m,n) in enumerate(matches):
    if m.distance < 0.85*n.distance:
        matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv.DrawMatchesFlags_DEFAULT)
img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
plt.imshow(img3,),plt.show()

(<matplotlib.image.AxesImage at 0x287cb24d180>, None)

In [28]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 100)
search_params = dict(checks = 50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)

# Guardar todos las buenas coincidencias según la verificación de Ratio de Lowe
good = []
for m,n in matches:
    if m.distance < 0.85*n.distance:
        good.append(m)
print(len(good))

10


In [29]:
MIN_MATCH_COUNT = 3

if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 1.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts, M)
    img2 = cv.polylines(img2, [np.int32(dst)], True, 255, 3, cv.LINE_AA)
else:
    print( "No se encontraron suficientes coincidencias - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None

print(mask)

[[0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]]


In [30]:
draw_params = dict(matchColor = (0,255,0), # dibujar coincidencias en verde
                   singlePointColor = None,
                   matchesMask = matchesMask, # dibujar solo inliers
                   flags = 2)
img3 = cv.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)
plt.imshow(img3, 'gray'),plt.show()

(<matplotlib.image.AxesImage at 0x287d356d030>, None)

Unificacion del codigo y ajustes de parametros

In [47]:
# ============================================================
# 1. CARGA DEL TEMPLATE Y PREPROCESAMIENTO
# ============================================================

template_path = './template/pattern.png'  # Ruta del template
img_template = cv.imread(template_path, 0)  # Cargar el template en escala de grises
if img_template is None:
    raise IOError("No se pudo cargar el template: " + template_path)

# Redimensionar el template (ajusta fx y fy según convenga)
img_template = cv.resize(img_template, (0, 0), fx=0.23, fy=0.23, interpolation=cv.INTER_AREA)
# Suavizar el template para reducir ruido
img_template = cv.GaussianBlur(img_template, (3, 3), 0)
# Invertir el template (opcional; en algunos casos mejora la detección)
img_template = 255 - img_template.copy()

# ============================================================
# 2. CONFIGURACIÓN DE SIFT CON PARÁMETROS AJUSTADOS
# ============================================================
# IMPORTANTE: Debes tener instalado "opencv-contrib-python" para usar cv.xfeatures2d.SIFT_create()
sift = cv.xfeatures2d.SIFT_create(nfeatures=5000, contrastThreshold=0.001)
# --> Parámetros: 
#     nfeatures=5000 (máximo número de keypoints a detectar)
#     contrastThreshold=0.01 (valores más bajos detectan más keypoints en zonas con poco contraste)

# ============================================================
# 3. CARGAR LA IMAGEN DESTINO (donde se buscan múltiples logos)
# ============================================================
img_path = './images/coca_multi.png'
img_color = cv.imread(img_path)
if img_color is None:
    raise IOError("No se pudo cargar la imagen: " + img_path)
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
# Suavizar la imagen destino para reducir el ruido
img_proc = cv.GaussianBlur(img_gray, (3, 3), 0)

# ============================================================
# 4. EXTRACCIÓN DE KEYPOINTS Y MATCHING CON FLANN
# ============================================================
# Extraer características del template y de la imagen destino
kp_template, des_template = sift.detectAndCompute(img_template, None)
kp_img, des_img = sift.detectAndCompute(img_proc, None)
print("Keypoints en el template:", len(kp_template))
print("Keypoints en la imagen destino:", len(kp_img))

# Configurar FLANN
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=300)  # Ajusta 'trees' para más precisión (150 aquí)
search_params = dict(checks=300)  # 'checks' determina cuántas veces se profundiza en la búsqueda
flann = cv.FlannBasedMatcher(index_params, search_params)

# Realizar knnMatch (k=2 para aplicar el Ratio Test)
matches = flann.knnMatch(des_template, des_img, k=2)

# Aplicar Ratio Test de Lowe para filtrar buenos matches (valor 0.9 es ajustable)
good_matches = [m for m, n in matches if m.distance < 0.9 * n.distance]
print("Número de good matches:", len(good_matches))

# ============================================================
# 5. CLUSTERIZACIÓN DE LOS MATCHES EN LA IMAGEN DESTINO USANDO DBSCAN
# ============================================================
# Obtén las coordenadas destino de cada good match
dst_coords = np.array([kp_img[m.trainIdx].pt for m in good_matches])
# Parámetros de DBSCAN: eps (distancia máxima para considerar vecinos) y min_samples (mínimo número de puntos por cluster)
# Ajusta estos valores según la densidad de los matches en tu imagen
dbscan = DBSCAN(eps=40, min_samples=3)  # eps=30 píxeles, min_samples=3
if len(dst_coords) > 0:
    clusters = dbscan.fit_predict(dst_coords)
else:
    clusters = np.array([])

print("Número de clusters encontrados (excluyendo ruido):", len(set(clusters)) - (1 if -1 in clusters else 0))

# ============================================================
# 6. PARA CADA CLUSTER, CALCULAR HOMOGRAFÍA Y DIBUJAR UN RECUADRO
# ============================================================
img_final = img_color.copy()

# Recorre cada cluster (ignora el ruido, que DBSCAN marca con -1)
unique_labels = set(clusters)
for label in unique_labels:
    if label == -1:
        continue  # Saltamos el ruido
    # Índices de matches en este cluster
    indices = np.where(clusters == label)[0]
    # Solo consideramos el cluster si tiene suficiente número de good matches
    if len(indices) < 4:  # Ahora usamos 4, ya que findHomography requiere al menos 4 puntos
        continue
    
    # Extraer los matches que pertenecen a este cluster
    cluster_matches = [good_matches[i] for i in indices]
    # Verificar nuevamente
    if len(cluster_matches) < 4:
        continue
    
    # Extraer los puntos del template (origen) y destino (imagen) para los matches del cluster
    src_pts = np.float32([kp_template[m.queryIdx].pt for m in cluster_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_img[m.trainIdx].pt for m in cluster_matches]).reshape(-1, 1, 2)
    
    # Calcular la homografía para este cluster usando RANSAC (umbral 1.0, ajustable)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 4.0)
    if M is None:
        continue
    h, w = img_template.shape  # Dimensiones del template
    pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
    dst = cv.perspectiveTransform(pts, M)
    # Dibujar el recuadro (polilínea) sobre la imagen final
    img_final = cv.polylines(img_final, [np.int32(dst)], True, (0, 255, 0), 3, cv.LINE_AA)

# ============================================================
# 7. MOSTRAR LA IMAGEN FINAL CON TODOS LOS RECUADROS DETECTADOS
# ============================================================
plt.figure(figsize=(12,8))
plt.imshow(cv.cvtColor(img_final, cv.COLOR_BGR2RGB))
plt.title("Detección múltiple de logos (clusters)")
plt.axis("off")
plt.show()


Keypoints en el template: 74
Keypoints en la imagen destino: 5000
Número de good matches: 23
Número de clusters encontrados (excluyendo ruido): 2


In [70]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = boxes.astype("float")
    pick = []

    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)

        xx1 = np.maximum(x1[last], x1[idxs[:-1]])
        yy1 = np.maximum(y1[last], y1[idxs[:-1]])
        xx2 = np.minimum(x2[last], x2[idxs[:-1]])
        yy2 = np.minimum(y2[last], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([len(idxs) - 1], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def detect_coca_cola_bottles(img_path, template_path):
    # Cargar las imágenes
    img = cv.imread(img_path)
    template = cv.imread(template_path)
    
    # Convertir a escala de grises
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    
    # Aplicar detección de bordes Canny al patrón
    template_edges = cv.Canny(template_gray, 50, 150)
    
    # Escalar el patrón a un tamaño fijo (0.2 del original)
    scale_width = 0.2
    scale_height = 0.2
    width = int(template_edges.shape[1] * scale_width)
    height = int(template_edges.shape[0] * scale_height)
    resized_template = cv.resize(template_edges, (width, height))
    
    # Aplicar Canny a la imagen de entrada
    img_edges = cv.Canny(img_gray, 150, 150)
    
    # Obtener dimensiones del template
    h, w = resized_template.shape
    
    # Aplicar template matching
    result = cv.matchTemplate(img_edges, resized_template, cv.TM_CCORR_NORMED)
    
    # Umbral de detección
    threshold = 0.22
    locations = np.where(result >= threshold)
    
    # Crear lista de cajas para aplicar NMS
    rects = []
    for pt in zip(*locations[::-1]):
        rects.append([pt[0], pt[1], pt[0] + w, pt[1] + h])
    
    rects = np.array(rects)
    final_boxes = non_max_suppression_fast(rects, 0.3)  # umbral de solapamiento
    
    # Dibujar cajas resultantes
    img_result = img.copy()
    for (x1, y1, x2, y2) in final_boxes:
        cv.rectangle(img_result, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Mostrar resultado
    plt.figure(figsize=(12, 8))
    plt.imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    plt.title(f"Método: TM_CCORR_NORMED con Canny\nDetecciones únicas: {len(final_boxes)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    return len(final_boxes), img_result

detections_count, result_img = detect_coca_cola_bottles('images/coca_multi.png', 'template/pattern.png')
print(f"Se detectaron {detections_count} botellas de Coca-Cola")


Se detectaron 8 botellas de Coca-Cola


# Punto 3

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from sklearn.cluster import DBSCAN

# ============================================================
# 1. CARGA DEL TEMPLATE Y PREPROCESAMIENTO
# ============================================================
template_path = './template/pattern.png'  # Ruta del template
img_template = cv.imread(template_path, 0)  # Cargar el template en escala de grises
if img_template is None:
    raise IOError("No se pudo cargar el template: " + template_path)

# Redimensionar el template (ajusta fx y fy según convenga)
img_template = cv.resize(img_template, (0, 0), fx=0.23, fy=0.23, interpolation=cv.INTER_AREA)
# Suavizar el template para reducir ruido
img_template = cv.GaussianBlur(img_template, (3, 3), 0)
# Invertir el template (opcional; en algunos casos mejora la detección)
img_template = 255 - img_template.copy()

# ============================================================
# 2. CONFIGURACIÓN DE SIFT CON PARÁMETROS AJUSTADOS
# ============================================================
# IMPORTANTE: Debes tener instalado "opencv-contrib-python" para usar cv.xfeatures2d.SIFT_create()
sift = cv.xfeatures2d.SIFT_create(nfeatures=5000, contrastThreshold=0.01)
# nfeatures=5000 -> Máximo número de keypoints a detectar.
# contrastThreshold=0.01 -> Umbral de contraste (valores menores detectan más puntos).

# ============================================================
# 3. OBTENER LA LISTA DE IMÁGENES EN LA CARPETA "images"
# ============================================================
image_files = glob.glob(os.path.join('./images', '*.jpg')) + glob.glob(os.path.join('./images', '*.png'))
if not image_files:
    raise IOError("No se encontraron imágenes en la carpeta 'images'.")

# ============================================================
# 4. PROCESAR CADA IMAGEN
# ============================================================
for img_path in image_files:
    print("Procesando:", img_path)
    # Cargar la imagen destino en color y obtener su versión en escala de grises
    img_color = cv.imread(img_path)
    if img_color is None:
        print("No se pudo cargar:", img_path)
        continue
    img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
    # Suavizar la imagen destino para reducir ruido
    img_proc = cv.GaussianBlur(img_gray, (3, 3), 0)
    
    # ============================================================
    # 5. EXTRAER KEYPOINTS Y DESCRIPTORES CON SIFT PARA TEMPLATE E IMAGEN DESTINO
    # ============================================================
    kp_template, des_template = sift.detectAndCompute(img_template, None)
    kp_img, des_img = sift.detectAndCompute(img_proc, None)
    print("  Keypoints - Template:", len(kp_template), "| Imagen:", len(kp_img))
    
    # ============================================================
    # 6. MATCHING CON FLANN Y RATIO TEST
    # ============================================================
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=300)  # Aumenta 'trees' para mayor precisión
    search_params = dict(checks=300)  # Mayor número de 'checks' para exactitud
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des_template, des_img, k=2)
    # Filtrar buenos matches usando Ratio Test (ajustable, aquí 0.9)
    good_matches = [m for m, n in matches if m.distance < 0.9 * n.distance]
    print("  Good matches:", len(good_matches))
    
    # ============================================================
    # 7. CLUSTERIZACIÓN DE LOS MATCHES CON DBSCAN
    # ============================================================
    # Obtener las coordenadas destino de cada good match
    dst_coords = np.array([kp_img[m.trainIdx].pt for m in good_matches])
    # Parámetros para DBSCAN:
    # - eps: distancia máxima (en píxeles) para considerar puntos vecinos.
    # - min_samples: mínimo número de puntos para formar un cluster.
    dbscan = DBSCAN(eps=50, min_samples=1)  # Ajusta 'eps' si se da un logo muy grande o pequeño.
    clusters = dbscan.fit_predict(dst_coords) if len(dst_coords) > 0 else np.array([])
    
    num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    print("  Número de clusters (instancias):", num_clusters)
    
    # ============================================================
    # 8. PARA CADA CLUSTER, CALCULAR HOMOGRAFÍA, DIBUJAR BOUNDING BOX Y MOSTRAR CONFIDENCIA
    # ============================================================
    img_final = img_color.copy()
    unique_labels = set(clusters)
    for label in unique_labels:
        if label == -1:  # DBSCAN marca con -1 el ruido
            continue
        # Obtén los índices de los matches en este cluster
        indices = np.where(clusters == label)[0]
        # Requerir al menos 4 matches para poder calcular homografía
        if len(indices) < 4:
            continue
        cluster_matches = [good_matches[i] for i in indices]
        # Extraer puntos correspondientes del template y de la imagen destino
        src_pts = np.float32([kp_template[m.queryIdx].pt for m in cluster_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_img[m.trainIdx].pt for m in cluster_matches]).reshape(-1, 1, 2)
        
        # Calcular homografía usando RANSAC. El umbral 1.0 es ajustable.
        M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 1.0)
        if M is None:
            continue
        # Calcular nivel de confianza como la proporción de inliers
        inliers_ratio = float(np.sum(mask)) / float(len(mask))
        
        h, w = img_template.shape  # Dimensiones del template
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst_box = cv.perspectiveTransform(pts, M)
        
        # Dibujar el bounding box (recuadro) sobre la imagen destino
        img_final = cv.polylines(img_final, [np.int32(dst_box)], True, (0, 255, 0), 3, cv.LINE_AA)
        # Obtener la esquina superior izquierda del recuadro para colocar el texto
        top_left = tuple(np.int32(dst_box[0][0]))
        # Escribir el nivel de confianza (porcentaje de inliers)
        cv.putText(img_final, f"{inliers_ratio*100:.1f}%", top_left, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # ============================================================
    # 9. MOSTRAR LA IMAGEN FINAL CON TODOS LOS BOUNDING BOXES Y CONFIDENCIA
    # ============================================================
    plt.figure(figsize=(12,8))
    plt.imshow(cv.cvtColor(img_final, cv.COLOR_BGR2RGB))
    plt.title("Detección múltiple de logos con bounding boxes y confianza\n" + os.path.basename(img_path))
    plt.axis("off")
    plt.show()

Procesando: ./images\COCA-COLA-LOGO.jpg
  Keypoints - Template: 74 | Imagen: 3879
  Good matches: 26
  Número de clusters (instancias): 22
Procesando: ./images\coca_logo_1.png
  Keypoints - Template: 74 | Imagen: 652
  Good matches: 45
  Número de clusters (instancias): 2
Procesando: ./images\coca_logo_2.png
  Keypoints - Template: 74 | Imagen: 726
  Good matches: 32
  Número de clusters (instancias): 4
Procesando: ./images\coca_multi.png
  Keypoints - Template: 74 | Imagen: 5000
  Good matches: 23
  Número de clusters (instancias): 13
Procesando: ./images\coca_retro_1.png
  Keypoints - Template: 74 | Imagen: 5000
  Good matches: 11
  Número de clusters (instancias): 10
Procesando: ./images\coca_retro_2.png
  Keypoints - Template: 74 | Imagen: 1862
  Good matches: 32
  Número de clusters (instancias): 5
Procesando: ./images\logo_1.png
  Keypoints - Template: 74 | Imagen: 776
  Good matches: 28
  Número de clusters (instancias): 7


In [120]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

def detectar_multiples_coca_logos():
    template_path = './template/pattern.png'
    image_path = './images/coca_multi.png'

    # Cargar imágenes
    img2_color = cv.imread(image_path)
    img2_gray = cv.cvtColor(img2_color, cv.COLOR_BGR2GRAY)

    img1 = cv.imread(template_path, 0)  # Template original
    img1 = cv.resize(img1, (0, 0), fx=0.4, fy=0.4, interpolation=cv.INTER_AREA)

    # Aplicar suavizado gaussiano
    img1 = cv.GaussianBlur(img1, (3, 3), 0)
    img2 = cv.GaussianBlur(img2_gray, (3, 3), 0)

    # Aplicar Canny a la imagen general (no al template)
    img2_trabajo = cv.Canny(img2, 200, 200)

    # Invertir template
    img1 = 255 - img1.copy()

    # Inicializar resultado final
    resultado = img2_color.copy()

    # Detectores
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    bf = cv.BFMatcher()

    print(f"Keypoints en template: {len(kp1)}")

    detecciones = 0
    MAX_ITERACIONES = 300

    for i in range(MAX_ITERACIONES):
        kp2, des2 = sift.detectAndCompute(img2_trabajo, None)
        if des2 is None or len(kp2) < 1:
            break

        matches = bf.knnMatch(des1, des2, k=2)
        good = [m for m, n in matches if m.distance < 0.8 * n.distance]

        print(f"Iteración {i+1}: {len(good)} good matches")

        if len(good) < 4:
            break

        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 3.0)
        inliers = np.sum(mask) if mask is not None else 0

        if M is not None and inliers >= 2:
            h, w = img1.shape
            box = np.float32([[0,0], [0,h], [w,h], [w,0]]).reshape(-1, 1, 2)
            dst = cv.perspectiveTransform(box, M)

            area = cv.contourArea(np.int32(dst))
            if area < 1000:
                print(f"Área muy pequeña: {area}")
                break

            resultado = cv.polylines(resultado, [np.int32(dst)], True, (0, 255, 0), 3, cv.LINE_AA)

            mask_poly = np.zeros_like(img2_trabajo)
            cv.fillPoly(mask_poly, [np.int32(dst)], 255)
            mask_poly = cv.dilate(mask_poly, np.ones((7, 7), np.uint8), iterations=1)
            img2_trabajo = cv.inpaint(img2_trabajo, mask_poly, 3, cv.INPAINT_TELEA)

            detecciones += 1
        else:
            break

    print(f"Detecciones totales: {detecciones}")

    plt.figure(figsize=(12, 8))
    plt.imshow(cv.cvtColor(resultado, cv.COLOR_BGR2RGB))
    plt.title(f"Detecciones proyectadas de logos (total: {detecciones})")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    return detecciones, resultado



In [121]:
detecciones, resultado = detectar_multiples_coca_logos()
cv.imwrite('resultado_coca_detecciones.png', resultado)
print(f"Imagen guardada con {detecciones} detecciones.")

Keypoints en template: 94
Iteración 1: 0 good matches
Detecciones totales: 0
Imagen guardada con 0 detecciones.
